# SQL Fundamentals: Using SQL with Polars

This notebook introduces SQL using Polars' built-in SQL functionality. No external database needed!

In [ ]:
# Import necessary libraries
import polars as pl

# Set display options
pl.Config.set_tbl_rows(10)
pl.Config.set_fmt_str_lengths(50)

print("Setup complete! Let's learn SQL with Polars.")

## 1. Loading Data and Creating SQL Context

First, let's load our NYC airports data and create a SQL context.

In [ ]:
# Read CSV files with Polars
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv')

# Display basic info about each dataset
print("Airlines:", airlines.shape)
print("Airports:", airports.shape)
print("Flights:", flights.shape)
print("Planes:", planes.shape)
print("Weather:", weather.shape)

In [ ]:
# Create SQL context and register our DataFrames
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True  # This makes queries return DataFrames instead of LazyFrames
)

print("SQL Context created! Tables available:")
print(ctx.execute("SHOW TABLES"))

## 2. Basic SQL: SELECT and FROM

The most fundamental SQL operation is selecting data from a table.

In [ ]:
# Polars: Select all columns
print("Polars - Select all columns:")
print(airlines.head(5))
print()

# SQL equivalent
print("SQL - Select all columns:")
result = ctx.execute("""
    SELECT * 
    FROM airlines
    LIMIT 5
""")
print(result)

In [ ]:
# Polars: Select specific columns
print("Polars - Select specific columns:")
print(airlines.select(['carrier', 'name']).head(5))
print()

# SQL equivalent
print("SQL - Select specific columns:")
result = ctx.execute("""
    SELECT carrier, name 
    FROM airlines
    LIMIT 5
""")
print(result)

## 3. Filtering Data: WHERE Clause

In SQL, we use WHERE to filter rows, similar to Polars' filter() method.

In [ ]:
# Polars: Filter flights from JFK
print("Polars - Flights from JFK:")
jfk_flights_polars = flights.filter(pl.col('origin') == 'JFK')
print(f"Number of JFK flights: {len(jfk_flights_polars)}")
print(jfk_flights_polars.head(5))
print()

# SQL equivalent
print("SQL - Flights from JFK:")
result = ctx.execute("""
    SELECT * 
    FROM flights
    WHERE origin = 'JFK'
    LIMIT 5
""")
print(result)

In [ ]:
# Multiple conditions
print("Polars - Delayed flights from JFK:")
delayed_jfk_polars = flights.filter(
    (pl.col('origin') == 'JFK') & 
    (pl.col('dep_delay') > 30)
)
print(f"Number of delayed JFK flights: {len(delayed_jfk_polars)}")
print()

# SQL equivalent
print("SQL - Delayed flights from JFK:")
result = ctx.execute("""
    SELECT COUNT(*) as count
    FROM flights
    WHERE origin = 'JFK' 
      AND dep_delay > 30
""")
print(result)

## 4. Sorting Data: ORDER BY

SQL uses ORDER BY to sort results, similar to Polars' sort() method.

In [ ]:
# Polars: Sort by departure delay
print("Polars - Most delayed flights:")
most_delayed_polars = flights.sort('dep_delay', descending=True).head(10)
print(most_delayed_polars.select(['carrier', 'flight', 'origin', 'dest', 'dep_delay']))
print()

# SQL equivalent
print("SQL - Most delayed flights:")
result = ctx.execute("""
    SELECT carrier, flight, origin, dest, dep_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    ORDER BY dep_delay DESC
    LIMIT 10
""")
print(result)

## 5. Aggregation: GROUP BY and Aggregate Functions

SQL aggregation uses GROUP BY with functions like COUNT, SUM, AVG, etc.

In [ ]:
# Polars: Average delay by carrier
print("Polars - Average delay by carrier:")
avg_delay_polars = (
    flights
    .group_by('carrier')
    .agg([
        pl.col('dep_delay').mean().alias('avg_delay'),
        pl.len().alias('flight_count')
    ])
    .sort('avg_delay', descending=True)
)
print(avg_delay_polars)
print()

# SQL equivalent
print("SQL - Average delay by carrier:")
result = ctx.execute("""
    SELECT 
        carrier,
        AVG(dep_delay) as avg_delay,
        COUNT(*) as flight_count
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY carrier
    ORDER BY avg_delay DESC
""")
print(result)

In [ ]:
# Multiple aggregations
print("SQL - Comprehensive carrier statistics:")
result = ctx.execute("""
    SELECT 
        carrier,
        COUNT(*) as total_flights,
        AVG(dep_delay) as avg_dep_delay,
        MAX(dep_delay) as max_dep_delay,
        MIN(dep_delay) as min_dep_delay,
        SUM(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) as delayed_flights
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY carrier
    HAVING COUNT(*) > 1000  -- Only carriers with more than 1000 flights
    ORDER BY avg_dep_delay DESC
""")
print(result)

## 6. Joining Tables

SQL joins are similar to Polars joins but with different syntax.

In [ ]:
# Polars: Join flights with airlines
print("Polars - Flights with airline names:")
flights_airlines_polars = (
    flights
    .join(airlines, on='carrier', how='left')
    .select(['carrier', 'name', 'flight', 'origin', 'dest'])
    .head(10)
)
print(flights_airlines_polars)
print()

# SQL equivalent
print("SQL - Flights with airline names:")
result = ctx.execute("""
    SELECT 
        f.carrier,
        a.name,
        f.flight,
        f.origin,
        f.dest
    FROM flights f
    LEFT JOIN airlines a ON f.carrier = a.carrier
    LIMIT 10
""")
print(result)

In [ ]:
# Multiple joins
print("SQL - Flights with airline and destination airport info:")
result = ctx.execute("""
    SELECT 
        f.flight,
        a.name as airline_name,
        f.origin,
        ap.name as destination_airport,
        ap.city as destination_city,
        f.dep_delay
    FROM flights f
    LEFT JOIN airlines a ON f.carrier = a.carrier
    LEFT JOIN airports ap ON f.dest = ap.faa
    WHERE f.dep_delay > 60
    ORDER BY f.dep_delay DESC
    LIMIT 10
""")
print(result)

## 7. Join Types Comparison

Let's explore different join types with a practical example.

In [ ]:
# Inner Join - Only matching records
print("INNER JOIN - Flights with plane info:")
result = ctx.execute("""
    SELECT COUNT(*) as flights_with_plane_info
    FROM flights f
    INNER JOIN planes p ON f.tailnum = p.tailnum
""")
print(result)

# Left Join - All flights, with plane info where available
print("\nLEFT JOIN - All flights:")
result = ctx.execute("""
    SELECT COUNT(*) as total_flights
    FROM flights f
    LEFT JOIN planes p ON f.tailnum = p.tailnum
""")
print(result)

# Flights without plane info
print("\nFlights without plane information:")
result = ctx.execute("""
    SELECT COUNT(*) as flights_no_plane_info
    FROM flights f
    LEFT JOIN planes p ON f.tailnum = p.tailnum
    WHERE p.tailnum IS NULL
""")
print(result)

## Bonus: Mixing SQL and Polars Operations

One advantage of using Polars SQL is that you can seamlessly mix SQL queries with Polars operations.

In [ ]:
# Start with SQL query
delayed_flights = ctx.execute("""
    SELECT *
    FROM flights
    WHERE dep_delay > 60
""")

# Continue with Polars operations
result = (
    delayed_flights
    .group_by(['origin', 'carrier'])
    .agg([
        pl.col('dep_delay').mean().alias('avg_delay'),
        pl.len().alias('count')
    ])
    .filter(pl.col('count') > 10)
    .sort('avg_delay', descending=True)
    .head(10)
)

print("Top origin-carrier combinations for severely delayed flights:")
print(result)

## Summary: SQL vs Polars Quick Reference

Here's a comparison of common operations:

| Operation | Polars | SQL |
|-----------|--------|-----|
| Select all | `df` | `SELECT * FROM table` |
| Select columns | `df.select(['col1', 'col2'])` | `SELECT col1, col2 FROM table` |
| Filter | `df.filter(pl.col('x') > 5)` | `WHERE x > 5` |
| Sort | `df.sort('col')` | `ORDER BY col` |
| Group by | `df.group_by('col').agg(...)` | `GROUP BY col` |
| Count | `pl.len()` | `COUNT(*)` |
| Mean | `pl.col('x').mean()` | `AVG(x)` |
| Sum | `pl.col('x').sum()` | `SUM(x)` |
| Join | `df1.join(df2, on='key')` | `FROM df1 JOIN df2 ON df1.key = df2.key` |

### Key Advantages of Polars SQL:
1. No external database needed
2. Seamlessly mix SQL and Polars operations
3. All queries are optimized by Polars' query engine
4. Results are always Polars DataFrames

Practice these concepts in the lab exercise!